In [1]:
from dmrApp import create_app, db
from dmrApp.models import Dmrs, Employees, Employeeroles, Post, Restaurants, Shifts, \
    User, Standardwages
import numpy as np
from datetime import date, datetime
from datetime import timedelta
from sqlalchemy import func
import pandas as pd
import os
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()

C:\tools\miniconda3\envs\dmrAppConda\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [77]:
ctx.pop()

In [80]:
dmr=db.session.query(Dmrs)

In [3]:
dmr.all()

[Dmr('1','2021-01-09','3','Tysons Biergarten'),
 Dmr('2','2021-01-09','1','CBC Restaurants'),
 Dmr('3','2021-01-09','2','Shipgarten'),
 Dmr('4','2021-01-09','3','Tysons Biergarten'),
 Dmr('6','2021-01-09','3','Tysons Biergarten'),
 Dmr('7','2021-01-12','3','Tysons Biergarten'),
 Dmr('8','2021-01-12','1','CBC Restaurants'),
 Dmr('9','2021-01-12','2','Shipgarten'),
 Dmr('10','2021-01-12','3','Tysons Biergarten'),
 Dmr('11','2021-01-12','3','Tysons Biergarten'),
 Dmr('12','2021-01-12','3','Tysons Biergarten'),
 Dmr('14','2021-01-12','3','Tysons Biergarten'),
 Dmr('15','2021-01-12','3','Tysons Biergarten'),
 Dmr('16','2021-01-12','6','Salamati Grill'),
 Dmr('17','2021-01-12','6','Salamati Grill'),
 Dmr('18','2021-01-12','6','Salamati Grill'),
 Dmr('19','2021-01-12','6','Salamati Grill'),
 Dmr('20','2021-01-12','6','Salamati Grill'),
 Dmr('21','2021-01-12','6','Salamati Grill'),
 Dmr('22','2021-01-12','6','Salamati Grill'),
 Dmr('23','2021-01-12','6','Salamati Grill')]

In [9]:
dateFrom=date(2021,1,1)
dateTo=date(2021,2,1)
resId=3
period=7

In [10]:
daysDelta=dateTo-dateFrom
daysDelta.days

31

In [81]:
shiftData=db.session.query(Shifts).filter(Shifts.shiftDate>=dateFrom,Shifts.shiftDate<=dateTo)

In [22]:
shiftData.all()

[Shift('1','','empId: 1','Nick','Tysons Biergarten'),
 Shift('2','','empId: 2','Shayan','Tysons Biergarten'),
 Shift('3','','empId: 3','apprentice1','Tysons Biergarten'),
 Shift('4','','empId: 3','apprentice1','Salamati Grill')]

In [72]:
shiftDf=pd.read_sql_table('shifts',db.engine)

In [30]:
tableNamesList= db.engine.table_names()
tableNamesList[5]

'shifts'

In [27]:
for i in db.Model._decl_class_registry.values():
    print(i)

<class 'dmrApp.models.Dmrs'>
<class 'dmrApp.models.Shifts'>
<class 'dmrApp.models.Restaurants'>
<class 'dmrApp.models.Employees'>
<class 'dmrApp.models.Employeeroles'>
<class 'dmrApp.models.Standardwages'>
<class 'dmrApp.models.User'>
<class 'dmrApp.models.Post'>


In [36]:
shiftDf

,id,shiftDate,name,schedTime,timeStart,timeOff,hoursWorked,shiftTips,shiftTipsShipgarten,wages,shiftsTimeStamp,empId,employeeRolesId,userId,restaurantId,restaurantGroupId
0,1,2021-01-09,Nick,,2021-01-09 09:00:00,2021-01-09 10:00:00,1.0,1.000000,1.000000,2.13,2021-01-12 08:45:03.052594,1,1,1,3,2
1,2,2021-01-12,Shayan,,2021-01-12 11:00:00,2021-01-12 14:00:00,3.0,0.333333,0.571429,6.39,2021-01-12 10:19:32.315499,2,3,1,3,2
2,3,2021-01-12,apprentice1,,2021-01-12 11:00:00,2021-01-12 23:00:00,12.0,0.666667,1.142857,25.56,2021-01-12 10:29:04.400358,3,5,1,3,2
3,4,2021-01-12,apprentice1,,2021-01-12 14:00:00,2021-01-12 17:00:00,3.0,1.000000,0.285714,6.39,2021-01-12 13:22:24.499063,3,5,1,6,2


In [40]:
str(dateFrom)

'2021-01-01'

In [43]:
#start by getting all shifts in the timeframe and restaurant
shiftDf[(shiftDf['shiftDate']>str(dateFrom)) &(shiftDf['shiftDate']<str(dateTo))]

,id,shiftDate,name,schedTime,timeStart,timeOff,hoursWorked,shiftTips,shiftTipsShipgarten,wages,shiftsTimeStamp,empId,employeeRolesId,userId,restaurantId,restaurantGroupId
0,1,2021-01-09,Nick,,2021-01-09 09:00:00,2021-01-09 10:00:00,1.0,1.000000,1.000000,2.13,2021-01-12 08:45:03.052594,1,1,1,3,2
1,2,2021-01-12,Shayan,,2021-01-12 11:00:00,2021-01-12 14:00:00,3.0,0.333333,0.571429,6.39,2021-01-12 10:19:32.315499,2,3,1,3,2
2,3,2021-01-12,apprentice1,,2021-01-12 11:00:00,2021-01-12 23:00:00,12.0,0.666667,1.142857,25.56,2021-01-12 10:29:04.400358,3,5,1,3,2
3,4,2021-01-12,apprentice1,,2021-01-12 14:00:00,2021-01-12 17:00:00,3.0,1.000000,0.285714,6.39,2021-01-12 13:22:24.499063,3,5,1,6,2


In [37]:
shiftDf.groupby(['empId']).mean()

,id,hoursWorked,shiftTips,shiftTipsShipgarten,wages,employeeRolesId,userId,restaurantId,restaurantGroupId
empId,,,,,,,,,
1,1.0,1.0,1.000000,1.000000,2.130,1.0,1.0,3.0,2.0
2,2.0,3.0,0.333333,0.571429,6.390,3.0,1.0,3.0,2.0
3,3.5,7.5,0.833333,0.714286,15.975,5.0,1.0,4.5,2.0


In [53]:
#table of empId and names no index
pd.unique(shiftDf['name']) #object array
pd.unique(shiftDf['name']).tolist() #object list

['Nick', 'Shayan', 'apprentice1']

In [62]:
shiftDf[shiftDf['name'].isin(pd.unique(shiftDf['name']).tolist())]

,id,shiftDate,name,schedTime,timeStart,timeOff,hoursWorked,shiftTips,shiftTipsShipgarten,wages,shiftsTimeStamp,empId,employeeRolesId,userId,restaurantId,restaurantGroupId
0,1,2021-01-09,Nick,,2021-01-09 09:00:00,2021-01-09 10:00:00,1.0,1.000000,1.000000,2.13,2021-01-12 08:45:03.052594,1,1,1,3,2
1,2,2021-01-12,Shayan,,2021-01-12 11:00:00,2021-01-12 14:00:00,3.0,0.333333,0.571429,6.39,2021-01-12 10:19:32.315499,2,3,1,3,2
2,3,2021-01-12,apprentice1,,2021-01-12 11:00:00,2021-01-12 23:00:00,12.0,0.666667,1.142857,25.56,2021-01-12 10:29:04.400358,3,5,1,3,2
3,4,2021-01-12,apprentice1,,2021-01-12 14:00:00,2021-01-12 17:00:00,3.0,1.000000,0.285714,6.39,2021-01-12 13:22:24.499063,3,5,1,6,2


In [68]:
nameEmpIdDf=shiftDf[['name','empId']].copy()
nameDfUnique=nameEmpIdDf.drop_duplicates()
meanDf=shiftDf.groupby(['empId']).mean()

In [67]:
nameDfUnique

,name,empId
0,Nick,1
1,Shayan,2
2,apprentice1,3


In [48]:
df.unqi

,name,empId
0,Nick,1
1,Shayan,2
2,apprentice1,3
3,apprentice1,3


In [85]:
#shifts in the timeframe and restaurant
shiftDf[(shiftDf['shiftDate']>str(dateFrom)) & (shiftDf['shiftDate']<str(dateTo)) & (shiftDf['restaurantId']==resId)]

,id,shiftDate,name,schedTime,timeStart,timeOff,hoursWorked,shiftTips,shiftTipsShipgarten,wages,shiftsTimeStamp,empId,employeeRolesId,userId,restaurantId,restaurantGroupId
0,1,2021-01-09,Nick,,2021-01-09 09:00:00,2021-01-09 10:00:00,1.0,1.000000,1.000000,2.130,2021-01-12 08:45:03.052594,1,1,1,3,2
1,2,2021-01-12,Shayan,,2021-01-12 11:00:00,2021-01-12 14:00:00,3.0,0.333333,0.571429,6.390,2021-01-12 10:19:32.315499,2,3,1,3,2
2,3,2021-01-12,apprentice1,,2021-01-12 11:00:00,2021-01-12 23:00:00,12.0,0.666667,1.142857,25.560,2021-01-12 10:29:04.400358,3,5,1,3,2
4,5,2021-01-14,apprentice1,,2021-01-14 07:00:00,2021-01-14 10:30:00,3.5,1.000000,1.000000,7.455,2021-01-14 06:29:09.462287,3,5,1,3,2


In [101]:
#get unique name and empId combinations for Df
nameEmpIdDf=shiftDf[['name','empId']].copy()
nameDfUnique=nameEmpIdDf.drop_duplicates()
#get mean
meanDf=shiftDf.groupby(['empId']).mean()
sumDf=shiftDf.groupby(['name','empId']).sum()

In [97]:
#table to make: sum hours, hourly rate,

#merge names and sum of hours
hoursDf=pd.merge(nameDfUnique,sumDf[['hoursWorked','shiftTips','shiftTipsShipgarten','wages']], on=['empId'])

In [102]:
week1Df=sumDf[['hoursWorked','shiftTips','shiftTipsShipgarten','wages']]
week1Df

,,hoursWorked,shiftTips,shiftTipsShipgarten,wages
name,empId,,,,
Nick,1,1.0,1.000000,1.000000,2.130
Shayan,2,3.0,0.333333,0.571429,6.390
apprentice1,3,18.5,2.666667,2.428571,39.405


In [237]:
#make list of weeks in loop
dateFrom=date(2021,1,3)
dateTo=date(2021,1,16)

weekStartDt=dateFrom
weekEndDt=dateFrom + timedelta(days=7)
print(weekStartDt, weekEndDt, dateTo, weekEndDt< dateTo)

2021-01-03 2021-01-10 2021-01-16 True


payroll report

In [179]:
#start by getting table into pandas
shiftDf=pd.read_sql_table('shifts',db.engine)
employeerolesDf=pd.read_sql_table('employeeroles',db.engine)
restaurantsDf=pd.read_sql_table('restaurants',db.engine)

In [264]:
resId=3
# dateFrom=date(2020,12,27)
# dateTo=date(2021,1,9)
dateFrom=date(2021,1,3)
dateTo=date(2021,1,16)
weekStartDt=dateFrom
weekEndDt=dateFrom + timedelta(days=6)
weeksDict={}
i=1
while weekEndDt< dateTo:
    weeksDict[f'week{i}']=[weekStartDt,weekEndDt]
    weekStartDt=weekEndDt + timedelta(days=1)
    weekEndDt = weekEndDt + timedelta(days=7)
    i+=1
# final week calculation
weeksDict[f'week{i}']=[weeksDict[f'week{i-1}'][0] + timedelta(days=7),weeksDict[f'week{i-1}'][1] + timedelta(days=7)]
weeksDict

{'week1': [datetime.date(2021, 1, 3), datetime.date(2021, 1, 9)],
 'week2': [datetime.date(2021, 1, 10), datetime.date(2021, 1, 16)]}

In [265]:
dfList=[]
i=1
for x,y in weeksDict.items():
    df=shiftDf[(shiftDf['shiftDate']>=str(y[0])) & (shiftDf['shiftDate']<=str(y[1])) & (shiftDf['restaurantId']==resId)]
    if len(df)>0:
        df=df.groupby(['name','empId','employeeRolesId','restaurantId']).sum()
        df=df[['hoursWorked','shiftTips','shiftTipsShipgarten','wages']].copy()
        df.columns=[f'Week{i} Hrs', f'Week{i} OtherRes Tips', f'Week{i} Shipgarten Tips', f'Week{i} Wages']
        dfList.append(df)
    i+=1



In [267]:
#get hourly rate from employee roles
rolesDf=employeerolesDf[['id','role','wage']].copy()
rolesDf.columns=['roleId','Role','Wage']
df2.reset_index(inplace=True)
df2.set_index('employeeRolesId')
df3=pd.merge(df2,rolesDf,how='left',left_on='employeeRolesId',right_on='roleId')


In [268]:
#Make columns
df3['Total Hours']=df3['Week1 Hrs']+df3['Week1 Hrs']
df3['Wage x Total Hours']=df3['Wage']*df3['Total Hours']
df3['Week1 Tips']=np.where((df3['restaurantId'] >=3) & (df3['restaurantId']<=6),df3['Week1 Shipgarten Tips'],df3['Week1 OtherRes Tips'])
df3['Week2 Tips']=np.where((df3['restaurantId'] >=3) & (df3['restaurantId']<=6),df3['Week2 Shipgarten Tips'],df3['Week2 OtherRes Tips'])
df3['Total Tips']=df3['Week1 Tips']+df3['Week2 Tips']

In [198]:
restaurantsDf.columns=['restaurantId','Restaurant Name']
df4=pd.merge(df3,restaurantsDf,how='left',left_on='restaurantId',right_on='restaurantId')

In [221]:
payrollReport=df4[['name','Restaurant Name','Role','Week1 Hrs','Week2 Hrs','Total Hours','Wage','Wage x Total Hours','Week1 Tips','Week2 Tips','Total Tips']].copy()
payrollReport.rename(columns={'name':'Name'},inplace=True)

In [238]:
reportName=f"Payroll Report {weeksDict['week1'][0].strftime('%m.%d')}-{weeksDict['week2'][1].strftime('%m.%d.%y')}.xlsx"
excelObj=pd.ExcelWriter(reportName)
payrollReport.to_excel(excelObj,sheet_name='payroll', startrow=1,header=False, index=False)
workbook=excelObj.book
worksheet=excelObj.sheets['payroll']
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'border': 0})

for col_num, value in enumerate(payrollReport.columns.values):
    worksheet.write(0, col_num, value,header_format)
    width=len(value) if len(value)>8 else 8
    worksheet.set_column(col_num,col_num,width)

formatDecimals = workbook.add_format({'num_format': '#,##0.00'})
worksheet.set_column('D:K', None, formatDecimals)
excelObj.close()

In [243]:
dateString='2020-12-27'
type(dateString)

str

In [247]:
import datetime
datetime.datetime.strptime(dateString,'%Y-%m-%d').date()

datetime.date(2020, 12, 27)